# Day 11: Payment Fraud Risk Detection in Online Transactions

You are a data analyst in Stripe's risk management team investigating transaction patterns to identify potential fraud. The team needs to develop a systematic approach to screen transactions for financial risks. Your goal is to create an initial risk assessment methodology using transaction characteristics.

In [ ]:
import pandas as pd
import numpy as np

dim_risk_flags_data = [
  {
    "risk_level": "Low",
    "risk_flag_id": 1,
    "transaction_id": 2
  },
  {
    "risk_level": "Medium",
    "risk_flag_id": 2,
    "transaction_id": 7
  },
  {
    "risk_level": "High",
    "risk_flag_id": 3,
    "transaction_id": 11
  },
  {
    "risk_level": "High",
    "risk_flag_id": 4,
    "transaction_id": 12
  },
  {
    "risk_level": "High",
    "risk_flag_id": 5,
    "transaction_id": 13
  },
  {
    "risk_level": "Medium",
    "risk_flag_id": 6,
    "transaction_id": 14
  },
  {
    "risk_level": "High",
    "risk_flag_id": 7,
    "transaction_id": 15
  },
  {
    "risk_level": "Low",
    "risk_flag_id": 8,
    "transaction_id": 1
  },
  {
    "risk_level": "Medium",
    "risk_flag_id": 9,
    "transaction_id": 6
  },
  {
    "risk_level": "Low",
    "risk_flag_id": 10,
    "transaction_id": 3
  }
]
dim_risk_flags = pd.DataFrame(dim_risk_flags_data)

fct_transactions_data = [
  {
    "customer_email": "alice@gmail.com",
    "transaction_id": 1,
    "transaction_date": "2024-10-05",
    "transaction_amount": 120,
    "fraud_detection_score": 10
  },
  {
    "customer_email": "bob@customdomain.com",
    "transaction_id": 2,
    "transaction_date": "2024-10-15",
    "transaction_amount": 250.5,
    "fraud_detection_score": 20
  },
  {
    "customer_email": "charlie@yahoo.com",
    "transaction_id": 3,
    "transaction_date": "2024-10-20",
    "transaction_amount": 75.25,
    "fraud_detection_score": 15
  },
  {
    "customer_email": "dana@hotmail.com",
    "transaction_id": 4,
    "transaction_date": "2024-10-25",
    "transaction_amount": 100,
    "fraud_detection_score": 30
  },
  {
    "customer_email": "eve@biz.org",
    "transaction_id": 5,
    "transaction_date": "2024-10-30",
    "transaction_amount": 300,
    "fraud_detection_score": 40
  },
  {
    "customer_email": "frank@gmail.com",
    "transaction_id": 6,
    "transaction_date": "2024-11-03",
    "transaction_amount": 150.75,
    "fraud_detection_score": 25
  },
  {
    "customer_email": "grace@outlook.com",
    "transaction_id": 7,
    "transaction_date": "2024-11-10",
    "transaction_amount": null,
    "fraud_detection_score": 50
  },
  {
    "customer_email": "ivan@yahoo.com",
    "transaction_id": 8,
    "transaction_date": "2024-11-15",
    "transaction_amount": 200,
    "fraud_detection_score": 35
  },
  {
    "customer_email": "judy@hotmail.com",
    "transaction_id": 9,
    "transaction_date": "2024-11-21",
    "transaction_amount": 250,
    "fraud_detection_score": 45
  },
  {
    "customer_email": "ken@domain.net",
    "transaction_id": 10,
    "transaction_date": "2024-11-29",
    "transaction_amount": 300,
    "fraud_detection_score": 55
  },
  {
    "customer_email": "laura@riskmail.com",
    "transaction_id": 11,
    "transaction_date": "2024-12-02",
    "transaction_amount": 100,
    "fraud_detection_score": 80
  },
  {
    "customer_email": "mike@securepay.com",
    "transaction_id": 12,
    "transaction_date": "2024-12-03",
    "transaction_amount": 180,
    "fraud_detection_score": 85
  },
  {
    "customer_email": "nina@trusthub.com",
    "transaction_id": 13,
    "transaction_date": "2024-12-09",
    "transaction_amount": 220,
    "fraud_detection_score": 90
  },
  {
    "customer_email": "oscar@fintech.com",
    "transaction_id": 14,
    "transaction_date": "2024-12-16",
    "transaction_amount": 140,
    "fraud_detection_score": 70
  },
  {
    "customer_email": "paula@alertsys.com",
    "transaction_id": 15,
    "transaction_date": "2024-12-23",
    "transaction_amount": 260,
    "fraud_detection_score": 95
  }
]
fct_transactions = pd.DataFrame(fct_transactions_data)


## Question 1

How many transactions in October 2024 have a customer email ending with a domain other than 'gmail.com', 'yahoo.com', or 'hotmail.com'? This metric will help us identify transactions associated with less common email providers that may indicate emerging risk patterns.

In [ ]:
import pandas as pd
import numpy as np

fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'], errors='coerce')

oct_2024 = fct_transactions[(fct_transactions['transaction_date'].dt.year == 2024) &
                            (fct_transactions['transaction_date'].dt.month == 10)].copy()

def extract_domain(email):
    if pd.isna(email):
        return np.nan
    email = str(email).strip().lower()
    parts = email.rsplit('@', 1)
    return parts[1] if len(parts) == 2 else np.nan

oct_2024['email_domain'] = oct_2024['customer_email'].apply(extract_domain)

common = {'gmail.com', 'yahoo.com', 'hotmail.com'}
oct_2024['is_uncommon_domain'] = ~oct_2024['email_domain'].isin(common)

uncommon_count = int(oct_2024['is_uncommon_domain'].sum())
print("Number of transactions in October 2024 with domains OTHER than gmail.com, yahoo.com, hotmail.com:", uncommon_count)

domain_breakdown = oct_2024.groupby('email_domain').size().reset_index(name='transaction_count') \
                           .sort_values('transaction_count', ascending=False).head(20)
print(domain_breakdown)

## Question 2

For transactions occurring in November 2024, what is the average transaction amount, using 0 as a default for any missing values? This calculation will help us detect abnormal transaction amounts that could be related to fraudulent activity.

In [ ]:
import pandas as pd
import numpy as np

fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'], errors='coerce')

nov_2024 = fct_transactions[
    (fct_transactions['transaction_date'].dt.year == 2024) &
    (fct_transactions['transaction_date'].dt.month == 11)
].copy()

nov_2024['transaction_amount'] = nov_2024['transaction_amount'].fillna(0)

avg_amount = nov_2024['transaction_amount'].mean()

summary_stats = nov_2024['transaction_amount'].agg(['count', 'min', 'max', 'mean'])

print(f"Average transaction amount for November 2024 (NaNs as 0): {avg_amount:.2f}")
print("\nSummary statistics for November 2024 transactions:")
print(summary_stats)

## Question 3

Among transactions flagged as 'High' risk in December 2024, which day of the week recorded the highest number of such transactions? This analysis is intended to pinpoint specific days with concentrated high-risk activity and support the development of our preliminary fraud detection score.

In [ ]:
import pandas as pd

fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'], errors='coerce')

high_risk_flags = dim_risk_flags[dim_risk_flags['risk_level'].str.lower() == 'high']

high_risk_txns = pd.merge(high_risk_flags, fct_transactions, on='transaction_id', how='inner')

dec_2024_high_risk = high_risk_txns[
    (high_risk_txns['transaction_date'].dt.year == 2024) &
    (high_risk_txns['transaction_date'].dt.month == 12)
].copy()

dec_2024_high_risk['weekday'] = dec_2024_high_risk['transaction_date'].dt.day_name()

weekday_counts = dec_2024_high_risk.groupby('weekday').size().reset_index(name='transaction_count')

top_weekday = weekday_counts.loc[weekday_counts['transaction_count'].idxmax()]

print("Weekday with the highest number of 'High' risk transactions in December 2024:")
print(top_weekday)

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)